In [8]:
! git remote add origin https://github.com/EonianCoda/continual_learning_forOD.git


Username for 'https://github.com': ^C


In [9]:
! git push -u origin main 

error: unknown option `EonianCoda'
usage: git push [<options>] [<repository> [<refspec>...]]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --repo <repository>   repository
    --all                 push all refs
    --mirror              mirror all refs
    -d, --delete          delete refs
    --tags                push tags (can't be used with --all or --mirror)
    -n, --dry-run         dry run
    --porcelain           machine-readable output
    -f, --force           force updates
    --force-with-lease[=<refname>:<expect>]
                          require old value of ref to be at this value
    --recurse-submodules (check|on-demand|no)
                          control recursive pushing of submodules
    --thin                use thin pack
    --receive-pack <receive-pack>
                          receive pack program
    --exec <receive-pack>
                          receive pack program
    -u, --set-upstream    set upstream for git 

In [4]:
! git config user.name "EonianCoda"
! git config user.email "textuseful15900@gmail.com"

In [1]:
import numpy as np
import os
from retinanet import model
from retinanet import coco_eval
from retinanet.dataloader import CocoDataset_inOrder, collater, Resizer, AspectRatioBasedSampler, Augmenter, Normalizer
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import collections
import torch
root_path = '/home/deeplab307/Documents/Anaconda/Shiang/CL/'
method = 'w_distillation'
data_split = '15+1'
start_round = 1
batch_size = 1

checkpoint_epcoh = 50

def checkDir(path):
    """check whether directory exists or not.If not, then create it 
    """
    if not os.path.isdir(path):
        os.mkdir(path)
def get_checkpoint_path(method, now_round, epoch):
    global root_path
    global data_split
    
    checkDir(os.path.join(root_path, 'model', method, 'round{}'.format(now_round)))
    checkDir(os.path.join(root_path, 'model', method, 'round{}'.format(now_round), data_split))
    
    path = os.path.join(root_path, 'model', method, 'round{}'.format(now_round), data_split,'voc_retinanet_{}_checkpoint.pt'.format(epoch))
    return path


def readCheckpoint(method, now_round, epoch, retinanet, optimizer = None, scheduler = None):
    print('readcheckpoint at Round{} Epoch{}'.format(now_round, epoch))
    prev_checkpoint = torch.load(get_checkpoint_path(method, now_round, epoch))
    retinanet.load_state_dict(prev_checkpoint['model_state_dict'])
    if optimizer != None:
        optimizer.load_state_dict(prev_checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(prev_checkpoint['scheduler_state_dict'])
    


coco_path = '/home/deeplab307/Documents/Anaconda/Shiang/CL/DataSet/VOC2012'



# dataset_train = CocoDataset_inOrder(coco_path, set_name='TrainVoc2012', dataset = 'voc',
#                                     transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]),
#                                    data_split=data_split, start_round=start_round)

# dataset_val = CocoDataset_inOrder(os.path.join(root_path, 'DataSet', 'VOC2012'), set_name="ValVoc2012", dataset = 'voc', 
#                 transform=transforms.Compose([Normalizer(), Resizer()]), 
#                 start_round=1, data_split = "20")
dataset_train = CocoDataset_inOrder(coco_path, set_name='TrainVoc2012', dataset = 'voc',
                                    transform=transforms.Compose([Normalizer(), Resizer()]),
                                   data_split=data_split, start_round=start_round)

# sampler = AspectRatioBasedSampler(dataset_train, batch_size = batch_size, drop_last=False)
# dataloader_train = DataLoader(dataset_train, num_workers=2, collate_fn=collater, batch_sampler=sampler)

retinanet = model.resnet50(num_classes=dataset_train.num_classes(), pretrained=True)
retinanet.cuda()

optimizer = optim.Adam(retinanet.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
loss_hist = collections.deque(maxlen=500)

readCheckpoint(method, start_round, checkpoint_epcoh, retinanet , optimizer, scheduler)

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
{'id': [[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 15, 16, 17, 18, 19], [6], [20], [14], [12], [13]], 'name': [['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person'], ['train'], ['sheep'], ['sofa'], ['pottedplant'], ['tvmonitor']]}
dataloader class_num = 15
readcheckpoint at Round1 Epoch50


In [27]:
import time
import pickle
retinanet.train()
retinanet.freeze_bn()

dataset = dataset_train

fail_id = []
losses = collections.defaultdict(list)
for idx, data in enumerate(dataset):
    start = time.time()
    try:
        optimizer.zero_grad()
        with torch.cuda.device(0):
            if torch.cuda.is_available():
                classification_loss, regression_loss = retinanet([data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0), data['annot'].cuda().unsqueeze(dim=0)])
            else:
                print('not have gpu')
                break

            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()

            img_id = dataset.image_ids[idx]


            classification_loss = float(classification_loss)
            regression_loss = float(regression_loss)
            loss = classification_loss + regression_loss

            losses[img_id] = [classification_loss, regression_loss, loss]

            optimizer.step()
            loss_hist.append(float(loss))

            #epoch_loss.append(float(loss))
            end = time.time()

            print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f} | Spend Time:{:1.2f}s'.format(
                                              '50', 
                                              idx, 
                                              float(classification_loss), 
                                              float(regression_loss), 
                                              np.mean(loss_hist),
                                              end - start))
            del classification_loss
            del regression_loss
    except Exception as e:
        print(e)
        fail_id.append(idx)
        continue

print('fail_id:',fail_id)


with open(os.path.join("/".join(get_checkpoint_path(method, 1, 50).split('/')[:-1]), 'losses.pickle'), 'wb') as f:
    pickle.dump(losses, f)
#print(losses)

Epoch: 50 | Iteration: 0 | Classification loss: 0.71616 | Regression loss: 0.46356 | Running loss: 0.63894 | Spend Time:0.07s
Epoch: 50 | Iteration: 1 | Classification loss: 0.45599 | Regression loss: 0.27079 | Running loss: 0.63963 | Spend Time:0.05s
Epoch: 50 | Iteration: 2 | Classification loss: 0.61635 | Regression loss: 0.38911 | Running loss: 0.64246 | Spend Time:0.05s
Epoch: 50 | Iteration: 3 | Classification loss: 0.39104 | Regression loss: 0.33380 | Running loss: 0.64310 | Spend Time:0.05s
Epoch: 50 | Iteration: 4 | Classification loss: 0.72215 | Regression loss: 0.38564 | Running loss: 0.64665 | Spend Time:0.05s
Epoch: 50 | Iteration: 5 | Classification loss: 0.13401 | Regression loss: 0.32161 | Running loss: 0.64520 | Spend Time:0.04s
Epoch: 50 | Iteration: 6 | Classification loss: 0.23653 | Regression loss: 0.23852 | Running loss: 0.64392 | Spend Time:0.05s
Epoch: 50 | Iteration: 7 | Classification loss: 0.25578 | Regression loss: 0.39032 | Running loss: 0.64394 | Spend Tim

Epoch: 50 | Iteration: 67 | Classification loss: 0.10868 | Regression loss: 0.07885 | Running loss: 0.64396 | Spend Time:0.04s
Epoch: 50 | Iteration: 68 | Classification loss: 0.58997 | Regression loss: 0.66985 | Running loss: 0.64712 | Spend Time:0.04s
Epoch: 50 | Iteration: 69 | Classification loss: 0.00520 | Regression loss: 0.04752 | Running loss: 0.64409 | Spend Time:0.04s
Epoch: 50 | Iteration: 70 | Classification loss: 0.01528 | Regression loss: 0.14548 | Running loss: 0.64163 | Spend Time:0.04s
Epoch: 50 | Iteration: 71 | Classification loss: 1.00784 | Regression loss: 0.47335 | Running loss: 0.64587 | Spend Time:0.04s
Epoch: 50 | Iteration: 72 | Classification loss: 0.14759 | Regression loss: 0.30031 | Running loss: 0.64488 | Spend Time:0.04s
Epoch: 50 | Iteration: 73 | Classification loss: 0.10783 | Regression loss: 0.24007 | Running loss: 0.64339 | Spend Time:0.04s
Epoch: 50 | Iteration: 74 | Classification loss: 0.77485 | Regression loss: 0.73906 | Running loss: 0.64772 | S

Epoch: 50 | Iteration: 133 | Classification loss: 0.83939 | Regression loss: 0.45150 | Running loss: 0.65447 | Spend Time:0.04s
Epoch: 50 | Iteration: 134 | Classification loss: 0.48121 | Regression loss: 0.71059 | Running loss: 0.65653 | Spend Time:0.04s
Epoch: 50 | Iteration: 135 | Classification loss: 0.46765 | Regression loss: 0.80592 | Running loss: 0.65888 | Spend Time:0.04s
Epoch: 50 | Iteration: 136 | Classification loss: 0.09267 | Regression loss: 0.15720 | Running loss: 0.65733 | Spend Time:0.04s
Epoch: 50 | Iteration: 137 | Classification loss: 0.56349 | Regression loss: 0.58697 | Running loss: 0.65920 | Spend Time:0.04s
Epoch: 50 | Iteration: 138 | Classification loss: 1.00943 | Regression loss: 0.17001 | Running loss: 0.66116 | Spend Time:0.04s
Epoch: 50 | Iteration: 139 | Classification loss: 0.89338 | Regression loss: 0.29375 | Running loss: 0.66314 | Spend Time:0.04s
Epoch: 50 | Iteration: 140 | Classification loss: 0.12699 | Regression loss: 0.29161 | Running loss: 0.6

Epoch: 50 | Iteration: 199 | Classification loss: 0.74801 | Regression loss: 0.33133 | Running loss: 0.68056 | Spend Time:0.04s
Epoch: 50 | Iteration: 200 | Classification loss: 0.06931 | Regression loss: 0.31044 | Running loss: 0.67964 | Spend Time:0.04s
Epoch: 50 | Iteration: 201 | Classification loss: 0.47120 | Regression loss: 0.55139 | Running loss: 0.68069 | Spend Time:0.04s
Epoch: 50 | Iteration: 202 | Classification loss: 0.37667 | Regression loss: 0.15387 | Running loss: 0.68023 | Spend Time:0.04s
Epoch: 50 | Iteration: 203 | Classification loss: 0.01574 | Regression loss: 0.13492 | Running loss: 0.67862 | Spend Time:0.04s
Epoch: 50 | Iteration: 204 | Classification loss: 0.04353 | Regression loss: 0.28333 | Running loss: 0.67756 | Spend Time:0.04s
Epoch: 50 | Iteration: 205 | Classification loss: 0.05397 | Regression loss: 0.21832 | Running loss: 0.67634 | Spend Time:0.04s
Epoch: 50 | Iteration: 206 | Classification loss: 0.21791 | Regression loss: 0.17229 | Running loss: 0.6

Epoch: 50 | Iteration: 266 | Classification loss: 0.45834 | Regression loss: 0.21516 | Running loss: 0.68168 | Spend Time:0.04s
Epoch: 50 | Iteration: 267 | Classification loss: 0.07545 | Regression loss: 0.14438 | Running loss: 0.68051 | Spend Time:0.04s
Epoch: 50 | Iteration: 268 | Classification loss: 0.34222 | Regression loss: 0.33847 | Running loss: 0.68051 | Spend Time:0.04s
Epoch: 50 | Iteration: 269 | Classification loss: 1.92543 | Regression loss: 0.68848 | Running loss: 0.68539 | Spend Time:0.04s
Epoch: 50 | Iteration: 270 | Classification loss: 0.40768 | Regression loss: 0.45705 | Running loss: 0.68585 | Spend Time:0.04s
Epoch: 50 | Iteration: 271 | Classification loss: 0.41575 | Regression loss: 0.46420 | Running loss: 0.68633 | Spend Time:0.04s
Epoch: 50 | Iteration: 272 | Classification loss: 0.79178 | Regression loss: 0.38642 | Running loss: 0.68757 | Spend Time:0.04s
Epoch: 50 | Iteration: 273 | Classification loss: 0.03389 | Regression loss: 0.08709 | Running loss: 0.6

...

Epoch: 50 | Iteration: 4874 | Classification loss: 0.23298 | Regression loss: 0.59364 | Running loss: 0.78932 | Spend Time:0.04s
Epoch: 50 | Iteration: 4875 | Classification loss: 0.01199 | Regression loss: 0.14429 | Running loss: 0.78710 | Spend Time:0.04s
Epoch: 50 | Iteration: 4876 | Classification loss: 1.19833 | Regression loss: 0.77690 | Running loss: 0.78542 | Spend Time:0.04s
Epoch: 50 | Iteration: 4877 | Classification loss: 0.33112 | Regression loss: 0.48586 | Running loss: 0.78520 | Spend Time:0.04s
Epoch: 50 | Iteration: 4878 | Classification loss: 0.10804 | Regression loss: 0.14748 | Running loss: 0.78382 | Spend Time:0.04s
Epoch: 50 | Iteration: 4879 | Classification loss: 0.04114 | Regression loss: 0.10781 | Running loss: 0.78217 | Spend Time:0.04s
Epoch: 50 | Iteration: 4880 | Classification loss: 0.10009 | Regression loss: 0.11244 | Running loss: 0.78204 | Spend Time:0.04s
Epoch: 50 | Iteration: 4881 | Classification loss: 0.58133 | Regression loss: 0.64087 | Running l

Epoch: 50 | Iteration: 4940 | Classification loss: 0.01485 | Regression loss: 0.15493 | Running loss: 0.78175 | Spend Time:0.04s
Epoch: 50 | Iteration: 4941 | Classification loss: 0.09792 | Regression loss: 0.43022 | Running loss: 0.78060 | Spend Time:0.04s
Epoch: 50 | Iteration: 4942 | Classification loss: 0.05989 | Regression loss: 0.26489 | Running loss: 0.78099 | Spend Time:0.04s
Epoch: 50 | Iteration: 4943 | Classification loss: 0.12192 | Regression loss: 0.26645 | Running loss: 0.77988 | Spend Time:0.04s
Epoch: 50 | Iteration: 4944 | Classification loss: 0.11959 | Regression loss: 0.13326 | Running loss: 0.77930 | Spend Time:0.04s
Epoch: 50 | Iteration: 4945 | Classification loss: 0.00610 | Regression loss: 0.06633 | Running loss: 0.77844 | Spend Time:0.04s
Epoch: 50 | Iteration: 4946 | Classification loss: 0.11561 | Regression loss: 0.27646 | Running loss: 0.77613 | Spend Time:0.04s
Epoch: 50 | Iteration: 4947 | Classification loss: 0.09420 | Regression loss: 0.25330 | Running l

Epoch: 50 | Iteration: 5005 | Classification loss: 0.04580 | Regression loss: 0.24079 | Running loss: 0.75339 | Spend Time:0.04s
Epoch: 50 | Iteration: 5006 | Classification loss: 1.97487 | Regression loss: 0.67617 | Running loss: 0.75671 | Spend Time:0.04s
Epoch: 50 | Iteration: 5007 | Classification loss: 0.13661 | Regression loss: 0.15107 | Running loss: 0.75708 | Spend Time:0.04s
Epoch: 50 | Iteration: 5008 | Classification loss: 0.51573 | Regression loss: 0.46371 | Running loss: 0.75830 | Spend Time:0.04s
Epoch: 50 | Iteration: 5009 | Classification loss: 0.31520 | Regression loss: 0.27692 | Running loss: 0.75810 | Spend Time:0.05s
Epoch: 50 | Iteration: 5010 | Classification loss: 0.52902 | Regression loss: 0.39795 | Running loss: 0.75953 | Spend Time:0.04s
Epoch: 50 | Iteration: 5011 | Classification loss: 0.01124 | Regression loss: 0.04723 | Running loss: 0.75923 | Spend Time:0.04s
Epoch: 50 | Iteration: 5012 | Classification loss: 0.09246 | Regression loss: 0.28292 | Running l

Epoch: 50 | Iteration: 5071 | Classification loss: 0.69723 | Regression loss: 0.37637 | Running loss: 0.75964 | Spend Time:0.04s
Epoch: 50 | Iteration: 5072 | Classification loss: 1.29324 | Regression loss: 0.45434 | Running loss: 0.75945 | Spend Time:0.04s
Epoch: 50 | Iteration: 5073 | Classification loss: 0.91727 | Regression loss: 0.67499 | Running loss: 0.76253 | Spend Time:0.04s
Epoch: 50 | Iteration: 5074 | Classification loss: 0.25237 | Regression loss: 0.34501 | Running loss: 0.76344 | Spend Time:0.04s
Epoch: 50 | Iteration: 5075 | Classification loss: 0.04351 | Regression loss: 0.12527 | Running loss: 0.76221 | Spend Time:0.04s
Epoch: 50 | Iteration: 5076 | Classification loss: 0.82200 | Regression loss: 0.25632 | Running loss: 0.75951 | Spend Time:0.04s
Epoch: 50 | Iteration: 5077 | Classification loss: 0.68412 | Regression loss: 0.51237 | Running loss: 0.76081 | Spend Time:0.05s
Epoch: 50 | Iteration: 5078 | Classification loss: 0.98377 | Regression loss: 0.43569 | Running l

Epoch: 50 | Iteration: 5135 | Classification loss: 0.00575 | Regression loss: 0.06539 | Running loss: 0.76165 | Spend Time:0.04s
Epoch: 50 | Iteration: 5136 | Classification loss: 0.00184 | Regression loss: 0.17320 | Running loss: 0.75871 | Spend Time:0.04s
Epoch: 50 | Iteration: 5137 | Classification loss: 0.29946 | Regression loss: 0.47055 | Running loss: 0.75422 | Spend Time:0.04s
Epoch: 50 | Iteration: 5138 | Classification loss: 0.00335 | Regression loss: 0.02347 | Running loss: 0.75253 | Spend Time:0.04s
Epoch: 50 | Iteration: 5139 | Classification loss: 0.38171 | Regression loss: 0.16017 | Running loss: 0.75295 | Spend Time:0.04s
Epoch: 50 | Iteration: 5140 | Classification loss: 0.33297 | Regression loss: 0.42688 | Running loss: 0.75298 | Spend Time:0.04s
Epoch: 50 | Iteration: 5141 | Classification loss: 0.07273 | Regression loss: 0.27843 | Running loss: 0.75290 | Spend Time:0.05s
Epoch: 50 | Iteration: 5142 | Classification loss: 0.04440 | Regression loss: 0.22215 | Running l

In [28]:
with open(os.path.join("/".join(get_checkpoint_path(method, 1, 50).split('/')[:-1]), 'losses.pickle'), 'rb') as f:
    losses = pickle.load(f)

In [31]:
losses

defaultdict(list,
            {2008006657: [0.716161847114563,
              0.4635643910121687,
              1.1797262381267317],
             2010005505: [0.45598888397216797,
              0.27079466057667584,
              0.7267835445488438],
             2008006663: [0.6163512468338013,
              0.3891051729317211,
              1.0054564197655225],
             2010005512: [0.3910413384437561,
              0.33380469482784547,
              0.7248460332716016],
             2010005511: [0.7221482992172241,
              0.38564046976438165,
              1.1077887689816057],
             2010005513: [0.13401061296463013,
              0.3216137092220177,
              0.45562432218664783],
             2008006667: [0.23653225600719452,
              0.23851639101961417,
              0.4750486470268087],
             2010005518: [0.2557826340198517,
              0.39032391157939467,
              0.6461065455992463],
             2010005522: [0.11753395944833755,
       

In [30]:
os.path.join("/".join(get_checkpoint_path(method, 1, 50).split('/')[:-1]), 'losses.pickle')

'/home/deeplab307/Documents/Anaconda/Shiang/CL/model/w_distillation/round1/15+1/losses.pickle'

In [37]:
losses = [v[2] for v in losses.values()]

[2699,
 4388,
 440,
 3593,
 3070,
 903,
 2985,
 5149,
 3697,
 4962,
 759,
 1984,
 2918,
 3670,
 4276,
 4480,
 5117,
 2049,
 1065,
 4818,
 2760,
 2179,
 2923,
 3111,
 4958,
 1162,
 3723,
 2110,
 781,
 4756,
 3363,
 4921,
 59,
 5138,
 3475,
 2054,
 5161,
 1885,
 2769,
 2846,
 4486,
 39,
 876,
 4141,
 2416,
 4917,
 2292,
 398,
 2816,
 3194,
 155,
 1345,
 838,
 683,
 3148,
 667,
 2280,
 2976,
 3756,
 4310,
 1811,
 3430,
 3909,
 2734,
 758,
 2880,
 2595,
 148,
 1785,
 180,
 2184,
 4717,
 936,
 64,
 3674,
 4573,
 4126,
 4429,
 3494,
 596,
 3703,
 213,
 4433,
 69,
 1430,
 4753,
 2341,
 719,
 1719,
 5013,
 359,
 4703,
 3892,
 292,
 4372,
 2551,
 1545,
 3093,
 207,
 2283,
 4230,
 1121,
 4496,
 5011,
 1135,
 1793,
 2323,
 1584,
 3058,
 1079,
 4778,
 2463,
 776,
 171,
 3804,
 1599,
 2078,
 1895,
 3530,
 4116,
 1504,
 3208,
 4547,
 2262,
 4247,
 1254,
 3075,
 364,
 4596,
 1978,
 425,
 1248,
 4028,
 2697,
 4200,
 920,
 1169,
 1140,
 2797,
 2555,
 5135,
 1838,
 1890,
 4945,
 521,
 3396,
 3133,
 1776

In [ ]:
method = 'w_distillation'

for i in range(50,51,10):
    print(np.mean(readCheckpoint(method, 1, i)))



In [ ]:
method = 'incremental'

for i in range(10,31,10):
    print(np.mean(readCheckpoint(method, 1, i)))

In [ ]:
method = 'incremental'

for i in range(10,51,10):
    print(np.mean(readCheckpoint(method, 0, i)))



In [ ]:
np.mean(readCheckpoint(method, 0, i)

In [ ]:
parts = []

for i in range(0,4):
    parts.append(retinanet.classificationModel.output.weight.data[i*9: i*9 + 9,:,:,:])

In [ ]:
import torch.nn as nn
output = nn.Conv2d(256, 9 * 20, kernel_size=3, padding=1)

In [ ]:
output.weight.data[0:9,:,:,:] = parts[3]
#vehicle(7)
for i in range(0,7):
    output.weight.data[9 + i*9:9 + i*9 + 9,:,:,:] = parts[0]
#furniture(6)
for i in range(0,6):
    output.weight.data[72 + i*9:72 + i*9 + 9,:,:,:] = parts[1]
#animals(6)
for i in range(0,6):
    output.weight.data[126 + i*9:126 + i*9 + 9,:,:,:] = parts[2]

In [ ]:
(output.weight.data[72:81,:,:,:] == output.weight.data[81:90,:,:,:]).any()

In [ ]:
import copy
prev_model = copy.deepcopy(retinanet)
retinanet.increase_class(1)

retinanet.cuda()
prev_model.cuda()

In [ ]:
import torch
test = torch.ones(2,256,30,30).cuda()

In [ ]:
prev_out = prev_model.classificationModel.output_act(prev_model.classificationModel.output(test))
cur_out = retinanet.classificationModel.output_act(retinanet.classificationModel.output(test))

In [ ]:

def change_shape1(out, num_classes):
    out1 = out.permute(0, 2, 3, 1)
    batch_size, width, height, channels = out1.shape
    out1 = out1.view(batch_size, width, height, 9, num_classes)
    
    return out1.contiguous().view(2, -1, num_classes)
def change_shape2(out, num_classes):
    out1 = out.permute(0, 2, 3, 1)
    batch_size, width, height, channels = out1.shape
    out1 = out1.view(batch_size, width, height,num_classes, 9)
    
    out1 = out1.permute(0, 1, 2, 4, 3)
    
    return out1.contiguous().view(2, -1, num_classes)
prev_out_new1 = change_shape1(prev_out, 19)
cur_out_new1 = change_shape1(cur_out, 20)

# prev_out_new2 = change_shape2(prev_out, 19)
# cur_out_new2 = change_shape2(cur_out, 20)
# (prev_out == cur_out[:,:171,:,:]).any()

In [ ]:
(prev_out_new1[:,:,:] == cur_out_new[:,:,:,:,:19]).all()

In [ ]:
(cur_out_new1 == cur_out_new2).all()

In [ ]:

(prev_out_new[:,:,:,:,:] == cur_out_new[:,:,:,:,:19]).all()

In [ ]:
(prev_out_new[:,:,:] == cur_out_new[:,:,:19]).all()

In [ ]:
(prev_out_new[:,:,:,:,:] == cur_out_new[:,:,:,:19,:]).all()

In [ ]:
cur_out_new[0,0,0,0,:19]

In [ ]:


(prev_out_new[0,0,0,0,:] == cur_out_new[0,0,0,0,:19]).all()


In [ ]:
(prev_out_new == cur_out_new[:,:,:,:171]).all()

In [ ]:
prev_out_new

In [ ]:
 == 

In [ ]:
cur_out_new.shape

In [ ]:
(cur_out_new[:,:,:,:171] == prev_out_new).all()

In [ ]:
import torch
import os
root_path = '/home/deeplab307/Documents/Anaconda/Shiang/CL/'
method = 'w_distillation'
start_round = 2
batch_size = 2

def get_checkpoint_path(method, now_round, epoch):
    global root_path

    path = os.path.join(root_path, 'model', method, 'round{}'.format(now_round), 'voc_retinanet_{}_checkpoint.pt'.format(epoch))
    return path

def readCheckpoint(method, now_round, epoch):
    prev_checkpoint = torch.load(get_checkpoint_path(method, now_round, epoch))
    return prev_checkpoint

In [ ]:
import numpy as np

In [ ]:
test = np.random.randn(5,2)
print(test)

In [ ]:
test[[1,2,3],]